# Read the input files

In [1]:
import pandas as pd

In [2]:
import glob
file_paths= glob.glob("../../../Datasets/*.csv")

### Code to just read a file quickly to check it

In [60]:
index=35
print(file_paths[index])
current= pd.read_csv(file_paths[index], encoding = "utf-8")

../../../Datasets\2003_04_26.csv


# All data cleaning

In [118]:
modified_current= {'Cumulative number of case(s) ': [int(value) for value in current['Cumulative number of case(s) ']],
                  'Number of deaths': [int(value) for value in current['Number of deaths']]}

In [ ]:
current.update(modified_current)
current

In [120]:
current.to_csv(file_paths[index], index= False)

In [ ]:
check= pd.read_csv(file_paths[index])
check

# Creating the JSON files


In [4]:
import json
import copy

In [5]:
relative_path= "../../"

In [ ]:
previous_json = {}
ignore= set(["../../../Datasets\\2003_07_31.csv"])

for index, file_path in enumerate(file_paths):
    print(file_path)

    if file_path in ignore:
        print("here")
        continue
    
    current= pd.read_csv(file_path, encoding = "utf-8")
           
    columns_list= list(current.columns)
    columns_dict= {}
  
    for index in range(len(columns_list)):
        temp= []
        for character in columns_list[index]:
            if character == " " or character.isalpha():
                temp.append(character)        
        columns_list[index]= "".join(temp)    
        columns_list[index]= columns_list[index].strip()
        columns_dict[columns_list[index]]= index
    
    current_json= {}    
    for row in current.iterrows():
        current_cases= int(row[1][columns_dict['Cumulative number of cases']])
        current_deaths= int(row[1][columns_dict['Number of deaths']])
        try:
            current_recovery= row[1][columns_dict['Number recovered']]
            try:
                current_recovery= int(current_recovery)
            except:
                current_recovery= 0
            
        except:
            current_recovery= 0
           
        current_country= {'cases': current_cases, 'deaths': current_deaths, 'recoveries': current_recovery}   
        
        # Clean country names
        country_name= row[1][0]
        temp= []
        for character in country_name:
            if character == " " or character.isalpha():
                temp.append(character)        
        country_name= "".join(temp)    
        country_name= country_name.strip()
        if "Hong Kong" in country_name:
            country_name= "Hong Kong"
        elif "Taiwan" in country_name:
            country_name= "Taiwan"
        elif "Macao" in country_name:
            country_name= "Macao"
        
        current_json[country_name]= current_country
    
    file_name_json= relative_path + "data/in/" + file_path[-14:-4] + ".json"  
    with open(file_name_json, 'w') as outfile:
        json.dump(current_json, outfile)    